In [1]:
# Import necessary libraries
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data_utils import *
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [2]:
# Define parameters
start_date = "2019-01-01"
end_date = "2024-06-30"
countries = ["AT", "BE", "CZ", "FR", "DE_LU", "HU","NL", "PL", "RO", "SK", "SI", "PL", "CH"]

# Load data
day_ahead_prices = load_day_ahead_prices(start_date, end_date, countries, analyze_missing=True)
generation_forecast = load_variable_data(start_date, end_date, "generation_forecast", countries, analyze_missing=True)
load_forecast = load_variable_data(start_date, end_date, "load_forecast", countries, analyze_missing=True)
wind_solar_forecast = load_variable_data(start_date, end_date, "wind_and_solar_forecast", countries, analyze_missing=True)
coal_gas_data = load_coal_gas_data(start_date, end_date)


Missing data analysis:
No missing values in the DataFrame

Missing data analysis:

FR:
  Total missing entries: 192
  Missing range: 2019-02-17 23:00:00+00:00 to 2019-02-18 22:00:00+00:00
  Missing range: 2021-02-19 23:00:00+00:00 to 2021-02-20 22:00:00+00:00
  Missing range: 2021-04-09 22:00:00+00:00 to 2021-04-12 21:00:00+00:00
  Missing range: 2021-05-19 22:00:00+00:00 to 2021-05-20 21:00:00+00:00
  Missing range: 2021-05-24 22:00:00+00:00 to 2021-05-25 21:00:00+00:00
  Missing range: 2021-05-27 22:00:00+00:00 to 2021-05-28 21:00:00+00:00
  Missing entries by year:
    2019: 24
    2021: 168

DE_LU:
  Total missing entries: 49
  Missing range: 2023-10-28 22:00:00+00:00 to 2023-10-28 22:00:00+00:00
  Missing range: 2024-06-20 22:00:00+00:00 to 2024-06-21 21:00:00+00:00
  Missing range: 2024-06-25 22:00:00+00:00 to 2024-06-26 21:00:00+00:00
  Missing entries by year:
    2023: 1
    2024: 48

PL:
  Total missing entries: 24
  Missing range: 2024-06-12 22:00:00+00:00 to 2024-06-13 21:

In [3]:
# Check consistency across all dataframes
print("\nChecking dataframe consistency:")
is_consistent = check_dataframe_consistency(
    day_ahead_prices, generation_forecast, load_forecast, wind_solar_forecast, verbose=True
)

if is_consistent:
    print("All dataframes are consistent in date range and dimensions.")
else:
    print("There are inconsistencies in the dataframes. Please review the output above for details.")

# Analyze missing data for coal_gas_data separately
print("\nAnalyzing missing data for coal and gas prices:")
analyze_missing_data(coal_gas_data)

# Print summary of missing data
print("\nSummary of missing data:")
for name, df in [
    ("Day-Ahead Prices", day_ahead_prices),
    ("Generation Forecast", generation_forecast),
    ("Load Forecast", load_forecast),
    ("Wind and Solar Forecast", wind_solar_forecast),
    ("Coal and Gas Prices", coal_gas_data)
]:
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n{name}:")
        print(missing[missing > 0])
    else:
        print(f"\n{name}: No missing values")

# Calculate percentage of missing data for each variable
total_datapoints = len(day_ahead_prices) * len(countries)
coal_gas_datapoints = len(coal_gas_data) * 2  # Assuming 2 columns for coal and gas

print("\nPercentage of missing data:")
print(f"Day-Ahead Prices: {day_ahead_prices.isnull().sum().sum() / total_datapoints:.2%}")
print(f"Generation Forecast: {generation_forecast.isnull().sum().sum() / total_datapoints:.2%}")
print(f"Load Forecast: {load_forecast.isnull().sum().sum() / total_datapoints:.2%}")
print(f"Wind and Solar Forecast: {wind_solar_forecast.isnull().sum().sum() / total_datapoints:.2%}")
print(f"Coal and Gas Prices: {coal_gas_data.isnull().sum().sum() / coal_gas_datapoints:.2%}")


Checking dataframe consistency:
No gaps in the time series data
No gaps in the time series data
No gaps in the time series data
All dataframes are consistent in date range and dimensions
All dataframes are consistent in date range and dimensions.

Analyzing missing data for coal and gas prices:

Missing data analysis:
No missing values in the DataFrame

Summary of missing data:

Day-Ahead Prices: No missing values

Generation Forecast:
FR       192
DE_LU     49
PL        24
RO        96
SK        72
SI       722
dtype: int64

Load Forecast:
DE_LU     49
HU        97
SI       215
dtype: int64

Wind and Solar Forecast:
FR    219
NL     24
PL     48
RO     72
SK    168
SI    580
dtype: int64

Coal and Gas Prices: No missing values

Percentage of missing data:
Day-Ahead Prices: 0.00%
Generation Forecast: 0.18%
Load Forecast: 0.06%
Wind and Solar Forecast: 0.18%
Coal and Gas Prices: 0.00%


In [4]:
generation_forecast = load_variable_data(start_date, end_date, "generation_forecast", countries)
wind_solar_forecast = load_variable_data(start_date, end_date, "wind_and_solar_forecast", countries)

# Check coinciding missing entries
coinciding_missing = (generation_forecast.isnull() & wind_solar_forecast.isnull()).sum()

print("Coinciding missing entries:")
print(coinciding_missing[coinciding_missing > 0])

# Check longest missing streaks
gen_streaks = longest_missing_streak(generation_forecast)
ws_streaks = longest_missing_streak(wind_solar_forecast)

print("\nLongest missing streaks for Generation Forecast:")
for country, (streak, start, end) in gen_streaks.items():
    if streak > 0:
        print(f"{country}: {streak} hours, from {start} to {end}")

print("\nLongest missing streaks for Wind and Solar Forecast:")
for country, (streak, start, end) in ws_streaks.items():
    if streak > 0:
        print(f"{country}: {streak} hours, from {start} to {end}")

# Check if the longest streaks overlap
for country in countries:
    gen_streak = gen_streaks[country]
    ws_streak = ws_streaks[country]
    if gen_streak[0] > 0 and ws_streak[0] > 0:
        gen_range = pd.date_range(gen_streak[1], gen_streak[2], freq='h')
        ws_range = pd.date_range(ws_streak[1], ws_streak[2], freq='h')
        overlap = gen_range.intersection(ws_range)
        if len(overlap) > 0:
            print(f"\nOverlapping missing data for {country}:")
            print(f"Overlap period: {overlap[0]} to {overlap[-1]}, {len(overlap)} hours")

Coinciding missing entries:
PL    24
SK    72
SI    73
dtype: int64

Longest missing streaks for Generation Forecast:
FR: 72 hours, from 2021-04-09 22:00:00+00:00 to 2021-04-12 21:00:00+00:00
DE_LU: 24 hours, from 2024-06-20 22:00:00+00:00 to 2024-06-21 21:00:00+00:00
PL: 24 hours, from 2024-06-12 22:00:00+00:00 to 2024-06-13 21:00:00+00:00
RO: 24 hours, from 2019-05-05 21:00:00+00:00 to 2019-05-06 20:00:00+00:00
SK: 48 hours, from 2019-06-15 22:00:00+00:00 to 2019-06-17 21:00:00+00:00
SI: 384 hours, from 2022-04-21 22:00:00+00:00 to 2022-05-07 21:00:00+00:00

Longest missing streaks for Wind and Solar Forecast:
FR: 48 hours, from 2020-07-11 22:00:00+00:00 to 2020-07-13 21:00:00+00:00
NL: 24 hours, from 2023-11-13 23:00:00+00:00 to 2023-11-14 22:00:00+00:00
PL: 24 hours, from 2019-02-10 23:00:00+00:00 to 2019-02-11 22:00:00+00:00
RO: 24 hours, from 2021-03-08 22:00:00+00:00 to 2021-03-09 21:00:00+00:00
SK: 48 hours, from 2019-06-15 22:00:00+00:00 to 2019-06-17 21:00:00+00:00
SI: 96 hou

## Compute Summary Statistics, bind into dataframe, and save to disk (for summary stats table)

In [5]:
# Load data
day_ahead_prices = load_day_ahead_prices(start_date, end_date, countries)
generation_forecast = load_variable_data(start_date, end_date, "generation_forecast", countries)
load_forecast = load_variable_data(start_date, end_date, "load_forecast", countries)
wind_solar_forecast = load_variable_data(start_date, end_date, "wind_and_solar_forecast", countries)
coal_gas_data = load_coal_gas_data(start_date, end_date)

def calculate_summary_stats(df, name, other_dfs=None):
    total_entries = df.size
    missing_entries = df.isnull().sum().sum()
    missing_percentage = missing_entries / total_entries
    longest_streaks = longest_missing_streak(df)
    max_streak = max(streak for streak, _, _ in longest_streaks.values())
    
    overlapping_missingness = 0
    if other_dfs:
        missing_mask = df.isnull()
        for other_df in other_dfs:
            overlapping_missingness += (missing_mask & other_df.isnull()).sum().sum()
    
    return pd.Series({
        'Total Entries': total_entries,
        'Missing Entries': missing_entries,
        'Missing Percentage': missing_percentage,
        'Longest Missing Streak': max_streak,
        'Overlapping Missingness': overlapping_missingness
    }, name=name)

# Calculate summary stats for each dataset
summary_stats = pd.DataFrame([
    calculate_summary_stats(day_ahead_prices, "Day-Ahead Prices", [generation_forecast, load_forecast, wind_solar_forecast]),
    calculate_summary_stats(generation_forecast, "Generation Forecast", [day_ahead_prices, load_forecast, wind_solar_forecast]),
    calculate_summary_stats(load_forecast, "Load Forecast", [day_ahead_prices, generation_forecast, wind_solar_forecast]),
    calculate_summary_stats(wind_solar_forecast, "Wind and Solar Forecast", [day_ahead_prices, generation_forecast, load_forecast]),
    calculate_summary_stats(coal_gas_data, "Coal and Gas Prices")
])

# Save the summary stats to parquet
summary_stats.to_parquet('../analysis/summary-stats/data_quality_summary.parquet')

print("Summary statistics saved to '../analysis/summary-stats/data_quality_summary.parquet'")
print(summary_stats)

Summary statistics saved to '../results/summary-stats/data_quality_summary.parquet'
                         Total Entries  Missing Entries  Missing Percentage  \
Day-Ahead Prices              578028.0              0.0            0.000000   
Generation Forecast           578028.0           1155.0            0.001998   
Load Forecast                 578028.0            361.0            0.000625   
Wind and Solar Forecast       578028.0           1111.0            0.001922   
Coal and Gas Prices            96338.0              0.0            0.000000   

                         Longest Missing Streak  Overlapping Missingness  
Day-Ahead Prices                            0.0                      0.0  
Generation Forecast                       384.0                    193.0  
Load Forecast                              25.0                    119.0  
Wind and Solar Forecast                    96.0                    264.0  
Coal and Gas Prices                         0.0                   

The "Overlapping Missingness" column represents the count of data points that are missing in both the current dataset and at least one other dataset.
A higher number indicates more instances where missing data in this dataset coincides with missing data in other datasets.

# Imputation of missing values

## Missing Data Imputation Method

### Overview

This method imputes missing values in panel data for electricity generation forecasts across multiple countries. It leverages the cross-sectional relationships between countries to estimate missing values.

### Key Features

1. **Panel Data Utilization**: Exploits the relationships between different countries' data.
2. **Temporal Consistency**: Processes data chronologically to ensure consistency and use of the most recent available information.
3. **Minimal Assumptions**: Doesn't impose fixed relationships between countries or assume any specific time series model.

### Methodology

1. For each day with missing data:
   a. Identify missing values for each country.
   b. For each missing value:
      - Use data from the same hour of the previous day to compute weights for other countries.
      - Calculate a weighted average of other countries' values for the missing time point.
      - Impute the missing value with this weighted average.

2. Weights are determined by the relative values of other countries in the previous day's corresponding hour.

3. Countries with missing data in the previous day are excluded from the calculation to ensure reliability.



### Advantages

- Adapts to changing relationships between countries over time.
- Preserves the relative influence of different countries.
- Handles missing data in a way that's consistent with the panel structure of the dataset.

### Limitations

- Assumes relative stability in cross-country relationships from one day to the next.
- May not capture longer-term trends or seasonal patterns.
- Effectiveness may be reduced if there are extended periods of missing data across multiple countries.

### Usage

```python
imputed_data = impute_missing_values(original_data)
```

This method provides a balance between leveraging available information and making minimal assumptions about the underlying data generation process.

In [6]:
# Generation, Load, and W&S Forecasts have missing values
imputed_generation_forecast = impute_missing_values(generation_forecast)
imputed_load_forecast = impute_missing_values(load_forecast)
imputed_wind_solar_forecast = impute_missing_values(wind_solar_forecast)

In [7]:
analyze_missing_data(imputed_generation_forecast)
analyze_missing_data(imputed_load_forecast)
analyze_missing_data(imputed_wind_solar_forecast)


Missing data analysis:
No missing values in the DataFrame

Missing data analysis:
No missing values in the DataFrame

Missing data analysis:
No missing values in the DataFrame


In [9]:
# Save
# save the three cleaned dataframes to parquet in the "cleaned" subfolder
os.makedirs('../data/cleaned', exist_ok=True)
imputed_generation_forecast.to_parquet('../data/cleaned/generation_forecast.parquet')
imputed_load_forecast.to_parquet('../data/cleaned/load_forecast.parquet')
imputed_wind_solar_forecast.to_parquet('../data/cleaned/wind_solar_forecast.parquet')

# Copy over the other datasets to the "cleaned" subfolder
day_ahead_prices.to_parquet('../data/cleaned/day_ahead_prices.parquet')
coal_gas_data.to_parquet('../data/cleaned/coal_gas_data.parquet')